In [ ]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def getAgodaReview(param):
    rs = rq.post(r'https://www.agoda.com/NewSite/en-us/Review/ReviewComments', json = param)
    soup = BeautifulSoup(rs.content, 'html.parser')
    comments = soup.find_all("div", attrs={"data-selenium":"individual-review-section"})

    result = []
    for comment in comments:
        title = comment.find(attrs={"data-selenium":"comments-title"}).text
        rate = comment.find(attrs={"data-selenium":"individual-review-rate"}).text
        review = comment.find(attrs={"data-selenium":"reviews-comments"})
        if review is not None:
            review = review.text.strip()
        result.append([title,rate,review])
    
    return result

In [ ]:
def createparam(hotelId,pageNum):
    param = {"hotelId":hotelId,"providerId":332,"demographicId":0,"page":pageNum,"pageSize":100,"sorting":1,"providerIds":332,
             "isReviewPage":"false","isCrawlablePage":"true","filters":{"language":[23],"room":[]},"searchKeyword":""}
    return param

In [ ]:
def getReviews(hotelId):
    currentPage = 1
    result = []
    while True:
        param = createparam(hotelId, currentPage)
        curr_req = getAgodaReview(param)
        result += curr_req
        currentPage += 1
        if len(curr_req) < 100:
            break
    
    df = pd.DataFrame(result, columns=['title','rating', 'reviews'])
    return df

In [ ]:
df_all_reviews = pd.read_excel("D:\\Data Science\\Hotel-Reviews-BM\\all-reviews.xlsx")

In [ ]:
df_all_reviews

In [ ]:
df_all_reviews['rating'].describe()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,10))
df_all_reviews['rating'].value_counts().sort_index().plot.bar()
plt.show()

In [ ]:
df_mediocre_rating = df_all_reviews[df_all_reviews['rating'] <= 5.0]
df_mediocre_rating.shape

In [ ]:
df_mediocre_rating

In [ ]:
df_reviews = df_all_reviews['reviews']

In [ ]:
import regex as rgx
def replaceShortForm(txt):
    txt = rgx.sub(r'([\.]+)|(\r\n)', ' ', txt.lower())
    txt = rgx.sub(r'\s(x)\s', ' tak ', txt)
    txt = rgx.sub(r'\sx([a-z]+)\s', r' tak \1 ', txt)
    return txt

df_reviews = df_reviews.apply(replaceShortForm)

In [ ]:
df_reviews

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
list(set(word_tokenize(df_reviews[0])))

In [ ]:
word_count = df_reviews.apply(lambda txt: pd.value_counts(word_tokenize(txt))) \
                       .sum(axis=0) \
                       .sort_values(axis=0, ascending=False)

In [ ]:
stopword2 = pd.read_csv(r'D:\Repositories\stopword.csv', index_col =0 )
stopword2 = list(stopword2['Stop_Word'].values)
stopword2 += [',','!','?']

In [ ]:
word_count_wo_stopword = word_count.drop(axis=0, labels=stopword2, errors='ignore')

In [ ]:
word_count_wo_stopword

In [ ]:
plt.figure(figsize=(10,10))
word_count_wo_stopword.head(10).plot.barh()
plt.show()

# Modeling of the Review

In [2]:
import pandas as pd
df_all_reviews = pd.read_excel("D:\\Data Science\\Hotel-Reviews-BM\\all-reviews-sent-only.xlsx")

In [3]:
df_all_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3086 entries, 0 to 3085
Data columns (total 2 columns):
Sentence    3086 non-null object
Polarity    1997 non-null object
dtypes: object(2)
memory usage: 48.3+ KB


In [4]:
df_with_Polarity = df_all_reviews.dropna(axis=0, subset=['Polarity'])

In [5]:
df_with_Polarity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1997 entries, 0 to 1998
Data columns (total 2 columns):
Sentence    1997 non-null object
Polarity    1997 non-null object
dtypes: object(2)
memory usage: 46.8+ KB


In [6]:
df_with_Polarity.head()

,Sentence,Polarity
0,"Sy tidak gembira, sy menginap di hotel 1 mlm u...",Negative
1,Sy terpaksa tanggung kos repair akibat bonet k...,Negative
2,"ini lah kali pertama, sy menginap di hotel yg ...",Negative
3,"sy harap pihak agoda, tidak bekerjasama dgn ho...",Negative
4,oklah,Neutral


In [7]:
df_with_Polarity.to_excel("D:\\Data Science\\Hotel-Reviews-BM\\sent-polarity.xlsx")

In [8]:
converted = pd.read_excel(r'D:\Data Science\word_list_airasia_converted.xls', sheet_name='Updated')
d = {}
for row in converted.itertuples():
    d[row[1]] = row[2]     

In [9]:
d

{'&': 'dan',
 'abgkat': 'abang dekat',
 'abis': 'habis',
 'ade': 'ada',
 'adoi': 'aduh',
 'adoiiii': 'aduh',
 'aerodarat': 'kapal darat',
 'agkt': 'angkat',
 'ahh': 'ah',
 'ailior': 'air liur',
 'airpot': 'lapangan terbang',
 'aje': 'sahaja',
 'ajelah': 'sahajalah',
 'ajer': 'sahaja',
 'ak': 'aku',
 'ambik': 'ambil',
 'amek': 'ambil',
 'amekla': 'ambil lah',
 'ana': 'saya',
 'angkt': 'angkat',
 'anual': 'tahunan',
 'apa2': 'apa-apa',
 'apapun': 'apa pun',
 'ape': 'apa',
 'arab': 'Arab',
 'area': 'kawasan',
 'aritu': 'hari itu',
 'ask': 'tanya',
 'awal2': 'awal-awal',
 'babi': 'khinzir',
 'badan2': 'badan-badan',
 'bag': 'beg',
 'baik2': 'baik-baik',
 'baling2': 'baling-baling',
 'bang': 'abang',
 'bangla': 'bangladesh',
 'banyk': 'banyak',
 'barang2': 'barang-barang',
 'bard': 'pujangga',
 'bargasi': 'bagasi',
 'basikal2': 'basikal-basikal',
 'bawak': 'bawa',
 'bawanges': 'bawang',
 'bdk2': 'budak-budak',
 'bebawang': 'bawang-bawang',
 'belagak': 'berlagak',
 'berat2': 'berat-berat',
 

In [10]:
from nltk.tokenize import word_tokenize, sent_tokenize
word_array = word_tokenize(df_with_Polarity['Sentence'][0])

In [11]:
d['&']

'dan'

In [12]:
word_array

['Sy',
 'tidak',
 'gembira',
 ',',
 'sy',
 'menginap',
 'di',
 'hotel',
 '1',
 'mlm',
 'utk',
 '2',
 'bilik',
 ',',
 'manager',
 'security',
 'sombong',
 'dan',
 'berlagak',
 '.']

In [13]:
for idx,word in enumerate(word_array):
    if word.isalpha() and word.lower() in d:
        word_array[idx] = d[word.lower()]

word_array

['saya',
 'tidak',
 'gembira',
 ',',
 'saya',
 'menginap',
 'di',
 'hotel',
 '1',
 'malam',
 'untuk',
 '2',
 'bilik',
 ',',
 'manager',
 'security',
 'sombong',
 'dan',
 'berlagak',
 '.']

In [14]:
import regex as rgx
class ReplaceWord(object):
    def __init__(self):
        self.shortform_dict = self.load_shortform_dict()
        self.english_dict = self.load_dictionary(r'D:\Data Science\Hotel-Reviews-BM\words_alpha.txt')
        self.bm_dict = self.load_dictionary(r'D:\Data Science\Hotel-Reviews-BM\dict-ms-wiki.txt')
        
        #; kata ganda has problem to detect whether number or words.
        self.kata_ganda = rgx.compile(r'(di|ke)?(\w+)2(kan|nya)?')
        self.kata_ganda_replacement = r'\1\2-\2\3'
    
    def load_shortform_dict(self):
        converted = pd.read_excel(r'D:\Data Science\word_list_airasia_converted.xls', sheet_name='Updated')
        d = {}
        for row in converted.itertuples():
            d[row[1]] = row[2]
        return d
    
    def load_dictionary(self, path):
        with open(path, 'r', encoding='utf-8') as dict_file:
            dict_array = dict_file.readlines()
        dict_file.close()
        return [rgx.sub(r'\n', '', word) for word in dict_array]
    
    def replace_multi_char(self, word):
        repeat_regexp = rgx.compile(r'(\w*)(\w)\2(\w*)')
        repl = r'\1\2\3'
        
        if word in self.bm_dict or word in self.english_dict:
            return word
        
        repl_word = repeat_regexp.sub(repl, word)
        if repl_word != word:
            return self.replace_multi_char(repl_word)
        else:
            return repl_word
 
    def check_replacement(self, word_array):
        # replace kata ganda
        word_array = [self.kata_ganda.sub(self.kata_ganda_replacement, word) for word in word_array]
        
        # replace words with multiple char
        word_array = [self.replace_multi_char(word) for word in word_array]
        
        # check for shortforms
        for idx,word in enumerate(word_array):
            is_kata_ganda = False
            if "-" in word and len(word)>=3:
                word = word.split("-")[0]
                is_kata_ganda = True

            if word.lower() in self.shortform_dict:
                converted_word = self.shortform_dict[word.lower()] + "-" + self.shortform_dict[word.lower()] if is_kata_ganda else self.shortform_dict[word.lower()] 
                word_array[idx] = converted_word

        # determine if malay words or english
        word_not_exist = []
        for word in word_array:
            if word.lower() not in self.english_dict and word.lower() not in self.bm_dict and word.isalpha():
                word_not_exist.append(word)
        
        return {'result': ' '.join(word_array), 'out_of_list': word_not_exist}

In [15]:
from nltk.tokenize import word_tokenize,sent_tokenize
class WordCleaner(object):  
    def remove_rep_non_chars(self, txt):
        pattern = rgx.compile(r'([\W_])\1{2,}')
        return pattern.sub(r'\1', txt)

    # pad fullstop with space, since tokenizer cannot detect dot if no space after it
    def remove_multi_dots(self, txt):
        pattern = rgx.compile(r'(\w)(\.{2,})')
        return pattern.sub(r'\1. ', txt)

    def pad_dot_with_space(self, txt):
        pattern = rgx.compile(r'(\w)(?:\s*)\.(\w)')
        return pattern.sub(r'\1. \2', txt)

    def remove_new_line(self, txt):
        pattern = rgx.compile(r'(\r|\n)?')
        return pattern.sub('', txt).strip()

    def kata_nafi(self, txt):            
        kata_nafi = rgx.compile(r'\b(?:x)(\w*)\b')
        kata_nafi_replacement = r'tak \1'
        return kata_nafi.sub(kata_nafi_replacement, txt)
    
    def replace_multi_dots(self, word):
        repeat_regexp = rgx.compile(r'(\w*)(\.)\2(\w*)')
        repl = r'\1\2\3'
        repl_word = repeat_regexp.sub(repl, word)
        if repl_word != word:
            return self.replace_multi_dots(repl_word)
        else:
            return repl_word
    
    def tokenize_sentence(self, paragraph_str):
        paragraph = self.clean_sentence(paragraph_str)
        sentences = sent_tokenize(paragraph)
        return sentences
    
    def clean_sentence(self, sent):
        return self.kata_nafi(self.pad_dot_with_space(self.remove_rep_non_chars(self.replace_multi_dots(sent))))

In [16]:
replacer = ReplaceWord()
cleaner = WordCleaner()

In [17]:
sample_str2 = 'terlalu tamak.... tu ja nak kata... try bila org bg... 3k sorang... Ambil la... Brg keperluan yg penting, susu anak, pampers,beras ...benda yg wajib setiap bulan beli.... Kan jimat... Kalau dpt kat aku... Beli susu n pampers anak. Jimat dah setahun.....'
result = [replacer.check_replacement(word_tokenize(sent_array))["result"] for sent_array in cleaner.tokenize_sentence(sample_str2)]

In [18]:
result

['terlalu tamak .',
 'itu sahaja nak kata .',
 'try bila orang bagi .',
 '3k seorang .',
 'Ambil lah .',
 'barang keperluan yang penting , susu anak , pampers , beras .',
 'benda yang wajib setiap bulan beli .',
 'Kan jimat .',
 'Kalau dapat dekat aku .',
 'Beli susu dan pampers anak .',
 'Jimat dah setahun .']

In [19]:
def normalize_sentence(sent):
    no_punct_sent = cleaner.clean_sentence(sent)
    #print(no_punct_sent)
    replaced_word = replacer.check_replacement(word_tokenize(no_punct_sent))['result']
    return replaced_word

In [20]:
normalize_sentence('& lokasi yang sesuai yang mana berhampiran dengan kl sentral dan jalan tidak berapa sesak di pagi hari')

'dan lokasi yang sesuai yang mana berhampiran dengan Kuala Lumpur sentral dan jalan tidak berapa sesak di pagi hari'

In [21]:
df_clean_sent = df_with_Polarity['Sentence'].apply(normalize_sentence)

In [22]:
df_with_Polarity = df_with_Polarity.assign(Clean_Sentence = df_clean_sent)

In [23]:
df_with_Polarity = df_with_Polarity[['Polarity', 'Sentence', 'Clean_Sentence']]

In [24]:
df_with_Polarity.to_excel("D:\\Data Science\\Hotel-Reviews-BM\\clean-polarity-sent.xlsx")

In [ ]:
import json
with open(r'D:\encycloDB\Dirty Words\DirtyWords.json', 'rb') as file:
    data = file.read()
dirty_words = json.loads(data.decode('utf-8'))

In [ ]:
en_dirty_words = [ word["word"] for word in dirty_words["RECORDS"] if word["language"] == 'en']

In [ ]:
len(en_dirty_words)

In [ ]:
df_dataset = pd.read_excel("D:\\Data Science\\Hotel-Reviews-BM\\clean-polarity-sent.xlsx")

In [ ]:
df_dataset.head()

In [ ]:
df_data_to_model = df_dataset[['Clean_Sentence','Polarity']]

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df_data_to_model, test_size=0.2, random_state=0)

In [ ]:
x_train=train_data['Clean_Sentence'].values
y_train=train_data['Polarity'].values

x_test=test_data['Clean_Sentence'].values
y_test=test_data['Polarity'].values

In [ ]:
tanya_list = ['kenapa','bila','siapa','mengapa','apa','bagaimana','berapa','mana']
perintah_list = ['jangan','sila','tolong','harap','usah','jemput','minta']
pangkal_list = ['maka','alkisah','arakian','syahdah','adapun','bermula','kalakian']
bantu_list = ['akan','telah','boleh','mesti','belum','sudah','dapat','masih','harus','hendak']
penguat_list = ['paling','agak','sungguh','amat','terlalu','nian','benar','paling']
penegas_list = ['jua','juga','sahaja','hanya','memang','lagi','pun']
nafi_list = ['bukan','tidak']
pemeri_list = ['ialah','adalah']
sendi_list = ['akan','kepada','terhadap','bagi','untuk','dari','daripada','di','dengan','hingga','sampai',
              'ke','kepada','oleh','pada','sejak','seperti','umpama','bak','tentang','laksanabagai',
             'semenjak','dalam','antara']
pembenar_list = ['ya','benar','betul']
nombor_list = ['satu','dua','tiga','empat','lima','enam','tujuh','lapan','sembilan','kosong']
suku_bilangan_list = ['per','suku','setengah','separuh','tiga suku']
pisahan_list = ['setiap','tiap']
keterangan_list = ['begitu','begini','demikian','perlahan','cepat','lena','akan','sedang','belum',
                   'telah','sekarang','sebentar','semalam','mungkin','agak','barangkali','pasti','tentu',
                   'sudah','selalu','kadang','acapkali','sesekali','yang']
arah_list = ['atas','bawah','tepi','antara','hadapan','utara','sisi','luar']
hubung_list = ['agar','apabila','atau','bahawa','dan','hingga','jika','jikalau','kecuali','kerana',
               'lalu','manakala','sambil','serta','semenjak','sementara','sungguhpun','supaya','walaupun','tetapi']
gantinama_list = ['aku','saya','hamba','patik','beta','kami','kita','anda','awak','engkau','tuanku','kalian',
                  'kamu','baginda','beliau','mereka']

# pos permulaan[:-4]
permulaan = ['bel','be','se','ter','men','memper','di','pe','me','ke','ber','pen','per']
# pos hujung [:1]
hujung = ['kan', 'kah','lah','tah','nya','an','wan','wati','ita']
alphabet = 'qwertyuiopasdfghjklzxcvbnm'

tatabahasa_dict = {'KT':tanya_list,'KP':perintah_list,'KPA':pangkal_list,'KB':bantu_list,'KPENGUAT':penguat_list,
                   'KPENEGAS':penegas_list,'KN':nafi_list, 'KPEMERI':pemeri_list,'KS':sendi_list,'KPEMBENAR':pembenar_list,
                   'NO':nombor_list,'SUKU':suku_bilangan_list,'PISAHAN':pisahan_list,'KETERANGAN':keterangan_list,
                   'ARAH':arah_list,'KH':hubung_list,'GN':gantinama_list}

stopword_tatabahasa = list(set(tanya_list+perintah_list+pangkal_list+bantu_list+penguat_list+\
                penegas_list+nafi_list+pemeri_list+sendi_list+pembenar_list+nombor_list+\
                suku_bilangan_list+pisahan_list+keterangan_list+arah_list+hubung_list+gantinama_list))

In [ ]:
stopword_tatabahasa

In [ ]:
with open(r'D:\Malaya\malaya\stop-word-kerulnet','r') as f:
    content = [x.strip() for x in f.readlines()]
    
stopwords = frozenset(content)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words = stopwords)

In [ ]:
train_data.shape

# Naive Bayes
## Before optimization

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([('vect', CountVectorizer(stop_words=stopwords)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf = text_clf.fit(x_train, y_train)

In [ ]:
import numpy as np
predicted = text_clf.predict(x_test)
np.mean(predicted == y_test)

In [ ]:
text_clf.predict(['Saya pasti akan kembali semula ke hotel ini.'])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted, digits=4))

## Finding best parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), \
              'clf__fit_prior': (True, False), 'clf__alpha':[0.1,0.5,0.8,1.0]}

gs_clf_NB = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf_NB = gs_clf_NB.fit(x_train, y_train)

In [ ]:
gs_clf_NB.best_score_
gs_clf_NB.best_params_

## Running with recommended parameters

In [ ]:
text_clf_cv = Pipeline([('vect', CountVectorizer(stop_words=stopwords, ngram_range=(1, 1))), ('tfidf', TfidfTransformer(use_idf=False)), ('clf', MultinomialNB(alpha=0.5, fit_prior=False))])
text_clf_cv = text_clf_cv.fit(x_train, y_train)

In [ ]:
predicted_cv = text_clf_cv.predict(x_test)
np.mean(predicted_cv == y_test)

In [ ]:
print(classification_report(y_test, predicted_cv, digits=4))

In [ ]:
text_clf_cv.predict(['Bilik yang disediakan agak kecil.'])

## Exporting Model

In [ ]:
from sklearn.externals import joblib
joblib.dump(text_clf_cv, 'sentiment-hotel.pkl') 

In [ ]:
clf = joblib.load('sentiment-hotel.pkl')

In [ ]:
clf.predict(['Check in mengambil masa terlalu lama.'])

In [ ]:
clf

# Logistic Classification

In [ ]:
# Stop Words
with open(r'D:\Malaya\malaya\stop-word-kerulnet','r') as f:
    content = [x.strip() for x in f.readlines()]
    
stopwords = frozenset(content)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
import numpy as np

stop_words=frozenset(stopwords)
text_clf_maxent = Pipeline([('vect', CountVectorizer(stop_words=stop_words, ngram_range=(1, 1))), ('tfidf', TfidfTransformer(use_idf = True)),
                         ('clf-svm', LogisticRegression(multi_class='multinomial', solver='saga'))])

text_clf_maxent = text_clf_maxent.fit(x_train, y_train)
predicted_maxent = text_clf_maxent.predict(x_test)
np.mean(predicted_maxent == y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), \
              'clf-svm__solver': ['newton-cg', 'sag', 'saga']}

gs_clf_maxEnt = GridSearchCV(text_clf_maxent, parameters, n_jobs=-1)
gs_clf_maxEnt = gs_clf_maxEnt.fit(x_train, y_train)

In [ ]:
gs_clf_maxEnt.best_score_
gs_clf_maxEnt.best_params_

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted_maxent, digits=4))

# SGD Classfiers

In [ ]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=500, 
                                                   tol=1e-2, random_state=42, n_jobs=-1, learning_rate='invscaling',
                                                  eta0=100))])

text_clf_svm = text_clf_svm.fit(x_train, y_train)
predicted_svm = text_clf_svm.predict(x_test)
np.mean(predicted_svm == y_test)

### TO DO

1. Parse time
2. dots with no space